# $\alpha$=0.1

In [1]:
import torch
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet18_Weights
import numpy as np
import torchvision.transforms as transforms        
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps

#  Preprocess
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# load pre-trained model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1).to(device)


# Temperature Scaling
subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)
model = ModelWithTemperature(model, temperature = 1.0).to(device)
model.set_temperature(train_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=4)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Before temperature - NLL: 1.337, ECE: 0.024
Optimal temperature: 0.989
After temperature - NLL: 1.337, ECE: 0.024


APS Classification, Start!

Running experiment 1/10...
q_hat = 0.9233927726745605
Total set size: 379336
Total coverage sets: 22530
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 15.17344
Average Coverage Rate in runs 1: 90.12%

Running experiment 2/10...
q_hat = 0.9238955140113831
Total set size: 381260
Total coverage sets: 22487
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 15.2504
Average Coverage Rate in runs 2: 89.95%

Running experiment 3/10...
q_hat = 0.9247675120830536
Total set size: 388329
Total coverage sets: 22627
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 15.53316
Average Coverage Rate in runs 3: 90.51%

Running experiment 4/10...
q_hat = 0.9196790277957917
Total set size: 362448
Total coverage sets: 22408
Total samples amount: 25000
Average Prediction Set Size Afte

# $\alpha$=0.2

In [2]:
# error rate
alpha = 0.2

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=4)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")



APS Classification, Start!

Running experiment 1/10...
q_hat = 0.821889066696167
Total set size: 160697
Total coverage sets: 19989
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 6.42788
Average Coverage Rate in runs 1: 79.96%

Running experiment 2/10...
q_hat = 0.8232402324676514
Total set size: 161441
Total coverage sets: 19954
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 6.45764
Average Coverage Rate in runs 2: 79.82%

Running experiment 3/10...
q_hat = 0.8268302083015442
Total set size: 167208
Total coverage sets: 20153
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 6.68832
Average Coverage Rate in runs 3: 80.61%

Running experiment 4/10...
q_hat = 0.8202929615974426
Total set size: 158388
Total coverage sets: 19897
Total samples amount: 25000
Average Prediction Set Size After APS in runs 4: 6.33552
Average Coverage Rate in runs 4: 79.59%

Running experiment 5/10...
q_hat = 0.82406824827194

# $\alpha$=0.05

In [3]:
# error rate
alpha = 0.05

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("\n")
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # split dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False,num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=4)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")



APS Classification, Start!

Running experiment 1/10...
q_hat = 0.9695879608392716
Total set size: 788473
Total coverage sets: 23773
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 31.53892
Average Coverage Rate in runs 1: 95.09%

Running experiment 2/10...
q_hat = 0.9695776134729385
Total set size: 787856
Total coverage sets: 23779
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 31.51424
Average Coverage Rate in runs 2: 95.12%

Running experiment 3/10...
q_hat = 0.9698827266693115
Total set size: 797180
Total coverage sets: 23801
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 31.8872
Average Coverage Rate in runs 3: 95.20%

Running experiment 4/10...
q_hat = 0.9679587453603744
Total set size: 758703
Total coverage sets: 23676
Total samples amount: 25000
Average Prediction Set Size After APS in runs 4: 30.34812
Average Coverage Rate in runs 4: 94.70%

Running experiment 5/10...
q_hat = 0.9691583067

## Result  
$\alpha$=0.1  
From the above test, following results can be collected :
- Final Average Prediction Set Size: **15.05**
- Final Average Coverage: **89.89%**  

$\alpha$=0.2  
From the above test, following results can be collected :
- Final Average Prediction Set Size: **6.54**
- Final Average Coverage: **80.06%**  

$\alpha$=0.05  
From the above test, following results can be collected :
- Final Average Prediction Set Size: **31.15**
- Final Average Coverage: **94.94%**